In [1]:
import pandas as pd 
from tqdm import tqdm
import os
import torch
import numpy as np
from torch.utils.data import Dataset
from torchvision import datasets, models, transforms
import PIL
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import classification_report

In [2]:
folder_path = 'nozzle_data'

In [7]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, csv_path, images_folder, transform = None):
        self.df = pd.read_csv(csv_path)
        self.images_folder = images_folder
        self.transform = transform

    def __len__(self):
        return len(self.df)
    def __getitem__(self, index):
        image_path = self.df.loc[index]['img_path']
        label = self.df.iloc[index][-1].astype(float)
        image = PIL.Image.open(os.path.join(self.images_folder, image_path))
        if self.transform is not None:
            image = self.transform(image)
        return image, label

In [8]:
images_folder = os.path.join(folder_path, 'images')
train_csv = os.path.join(folder_path,'train.csv')

transform = transforms.Compose([
                    transforms.AutoAugment(),
                    transforms.ToTensor(),
                    transforms.Resize(size=(480, 640), antialias=True),
                    transforms.RandomHorizontalFlip(),
                    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
dataset = CustomDataset(train_csv, images_folder,transform)
trainDataset,testDataset=torch.utils.data.random_split(dataset, [0.9,0.1],)

In [9]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [10]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [14]:


model = models.resnet18(weights='ResNet18_Weights.IMAGENET1K_V1')
model_features = model.fc.in_features
params = model.parameters()
    
model.fc = nn.Sequential(
    nn.Linear(model_features, 256),
    nn.Linear(256, 128),
    nn.Linear(128, 64),
    nn.Linear(64, 8),
    nn.Linear(8, 2)
)
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [17]:
train_loader =  DataLoader(trainDataset, batch_size=32, shuffle=True)
test_loader =  DataLoader(testDataset, batch_size=32, shuffle=False)
model.train(mode=True)
size = len(trainDataset)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4, weight_decay=1e-4, momentum=0.9)

epochs = 13

In [18]:
model.train(mode=True)
for epoch in range(epochs):
    running_loss = 0.0
    total = 0
    correct = 0
    progress_bar = tqdm(range(size//32), position=0, desc='[TRAINING]. ')
    train_loss_history = []
    for i, data in enumerate(train_loader):
        inputs, labels = data
        labels = labels.type(torch.LongTensor)
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        labels = labels  
        correct += (predicted == labels).sum().item()
        accuracy = correct / total * 100
        progress_bar.set_description(f"EPOCH [{epoch+1}/{epochs}]")
        progress_bar.set_postfix(loss=loss, acc=(100*correct/total))
        progress_bar.update()
        
    print(f'epoch: {epoch + 1} loss: {running_loss / len(train_loader):.3f}, accuracy: {accuracy:.2f}%')
    running_loss = 0.0
    total = 0
    correct = 0

print('Finished Training')

torch.save(model.state_dict(), 'model_parameters_2.pth')

EPOCH [1/10]:   2%|█▏                                                     | 49/2279 [00:56<43:07,  1.16s/it, acc=43.1, loss=tensor(0.6987, device='cuda:0', grad_fn=<NllLossBackward0>)]
/tmp/ipykernel_472551/1188216517.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  label = self.df.iloc[index][-1].astype(float)
EPOCH [1/13]: : 2280it [33:59,  1.14it/s, acc=55.9, loss=tensor(0.6349, device='cuda:0', grad_fn=<NllLossBackward0>)]                                                                   

epoch: 1 loss: 0.669, accuracy: 55.92%


EPOCH [1/13]: : 2280it [33:59,  1.12it/s, acc=55.9, loss=tensor(0.6349, device='cuda:0', grad_fn=<NllLossBackward0>)]                                          | 0/2279 [00:00<?, ?it/s]
EPOCH [2/13]: : 2280it [35:40,  1.19it/s, acc=82.1, loss=tensor(0.1469, device='cuda:0', grad_fn=<NllLossBackward0>)]                                                                   

epoch: 2 loss: 0.413, accuracy: 82.06%


EPOCH [2/13]: : 2280it [35:40,  1.07it/s, acc=82.1, loss=tensor(0.1469, device='cuda:0', grad_fn=<NllLossBackward0>)]                                          | 0/2279 [00:00<?, ?it/s]
EPOCH [3/13]: : 2280it [36:39,  1.00it/s, acc=94.9, loss=tensor(0.0501, device='cuda:0', grad_fn=<NllLossBackward0>)]                                                                   

epoch: 3 loss: 0.131, accuracy: 94.87%


EPOCH [3/13]: : 2280it [36:39,  1.04it/s, acc=94.9, loss=tensor(0.0501, device='cuda:0', grad_fn=<NllLossBackward0>)]                                          | 0/2279 [00:00<?, ?it/s]
EPOCH [4/13]: : 2280it [37:33,  1.02s/it, acc=97.2, loss=tensor(0.0068, device='cuda:0', grad_fn=<NllLossBackward0>)]                                                                   

epoch: 4 loss: 0.075, accuracy: 97.17%


EPOCH [4/13]: : 2280it [37:33,  1.01it/s, acc=97.2, loss=tensor(0.0068, device='cuda:0', grad_fn=<NllLossBackward0>)]                                          | 0/2279 [00:00<?, ?it/s]
EPOCH [5/13]:   0%|▏                                                       | 9/2279 [00:09<40:19,  1.07s/it, acc=98.6, loss=tensor(0.0334, device='cuda:0', grad_fn=<NllLossBackward0>)]

KeyboardInterrupt: 

In [10]:
#model.load_state_dict(torch.load('model_parameters.pth'))

C:\Users\aalqa\AppData\Local\Temp\ipykernel_18356\3996904583.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('model_parameters.pth'))


<All keys matched successfully>

In [19]:
model.eval()

correct = 0
total = 0
all_labels = []
all_preds = []
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        real = labels
        print(predicted)
        total += labels.size(0)
        all_labels.extend(real.cpu().numpy())
        all_preds.extend(predicted.cpu().numpy())
        correct += (predicted == real).sum().item()

        accuracy = correct / total * 100
        print(f'Test Accuracy: {accuracy:.2f}%') 

/tmp/ipykernel_472551/1188216517.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  label = self.df.iloc[index][-1].astype(float)


tensor([0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1,
        1, 1, 0, 1, 1, 0, 1, 0], device='cuda:0')
Test Accuracy: 100.00%
tensor([0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0,
        0, 0, 0, 1, 0, 0, 1, 0], device='cuda:0')
Test Accuracy: 100.00%
tensor([1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0,
        0, 0, 1, 1, 1, 0, 0, 1], device='cuda:0')
Test Accuracy: 100.00%
tensor([1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0,
        1, 1, 1, 0, 0, 0, 0, 0], device='cuda:0')
Test Accuracy: 100.00%
tensor([0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1,
        0, 0, 0, 1, 0, 0, 0, 0], device='cuda:0')
Test Accuracy: 99.38%
tensor([0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1,
        1, 1, 1, 1, 0, 1, 1, 0], device='cuda:0')
Test Accuracy: 99.48%
tensor([0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1,
    

In [ ]:
torch.save(model.state_dict(), 'model_parameters_2.pth')